In [54]:
import glob
import pandas as pd
from os import scandir
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [55]:
# MAC

# name_files = scandir("names")
# df = pd.DataFrame()
# i = 0
# for file in name_files:
#     year = file.name[3:7]
#     with open(f"names/{file.name}", "r") as handle:
#         lines = handle.read().splitlines()
#     names = [[year] + line.split(",") for line in lines]
#     df = df.join(pd.DataFrame(names, columns=["year", "name", "gender", "populairity"]))
#     i += 1
#     if i == 2:
#         break
# df


In [56]:
# WINDOWS

name_files = scandir("names")
df = pd.DataFrame()
all_data = pd.DataFrame()
for file in glob.glob("./names/*.txt"):
    df = pd.read_csv(file, sep=',', header=None)
    df["year"] = int(file[11:15])
    all_data = pd.concat([all_data, df])
df = all_data
df.columns = ["name", "sex", "count", "year"]
print(df)


            name sex  count  year
0           Mary   F   7065  1880
1           Anna   F   2604  1880
2           Emma   F   2003  1880
3      Elizabeth   F   1939  1880
4         Minnie   F   1746  1880
...          ...  ..    ...   ...
32028      Zylas   M      5  2018
32029      Zyran   M      5  2018
32030      Zyrie   M      5  2018
32031      Zyron   M      5  2018
32032      Zzyzx   M      5  2018

[1957046 rows x 4 columns]


In [76]:
filtr_09_18_M = (df.year <= 2018) & (df.year >= 2009) & (df.sex == 'M')
filtr_09_18_F = (df.year <= 2018) & (df.year >= 2009) & (df.sex == 'F')
popular_F_09_18 = df.loc[filtr_09_18_F]
popular_M_09_18 = df.loc[filtr_09_18_M]
print(df.loc[filtr_09_18_M, df.columns != "year"].groupby(
    "name").sum().sort_values(by="count", ascending=False))
print(df.loc[filtr_09_18_F, df.columns != "year"].groupby(
    "name").sum().sort_values(by="count", ascending=False))


            count
name             
Noah       180904
Jacob      173195
William    163811
Liam       161558
Ethan      157359
...           ...
Javere          5
Thorbjorn       5
Javez           5
Dailey          5
Sayhan          5

[25604 rows x 1 columns]
                count
name                 
Emma           195305
Sophia         183930
Olivia         183016
Isabella       179015
Ava            156852
...               ...
Naiima              5
Aramide             5
Elizabethanne       5
Elizabel            5
Kayleemarie         5

[34967 rows x 1 columns]


In [77]:
fig = px.line(popular_M_09_18.loc[(popular_M_09_18["name"].isin(['Noah', 'Jacob', 'William', 'Liam', 
'Ethan']))], x="year", y="count", color="name", title="Popular boys names 2009-2018")
fig.show()

In [78]:
fig = px.line(df.loc[(df["sex"] == "M") & (df["name"].isin(['Noah', 'Jacob', 'William', 'Liam',
                                                            'Ethan']))], x="year", y="count", color="name", title="Popular boys names")
fig.show()

To clearly see trends in the popularity of names, it is better to look at a wider range of years.
We have plotted the 5 most popular names from 2009 to 2018. The name William was very popular throughout the years. The other names gained popularity around 90s, with the exception of the name Jacob, which began to gain popularity as early as 70s. Currently, the name with the greatest increase in popularity is the name Liam. 

In [79]:
fig = px.line(popular_F_09_18.loc[(popular_F_09_18["name"].isin(
    ['Emma', 'Sophia', 'Olivia', 'Isabella', 'Ava']))], x="year", y="count", color="name", title="Popular girls names 2009-2018")
fig.show()

In [85]:
fig = px.line(df.loc[(df["sex"] == "F") & (df["name"].isin(
    ['Emma', 'Sophia', 'Olivia', 'Isabella', 'Ava']))], x="year", y="count", color="name", title="Popular girls names")
fig.show()


When it comes to names for girls (top 5 from 2009-2018), there is no name as popular as the boy's name William. Only the name Emma stands out, as it was the name of many more girls than other names until the 1960s. All of the top 5 names began to gain popularity in 90s. It is visible more than in case of boys, that girl names have peaks of popularity. 
Emma around 2000, Isabella around 2010, Sophia around 2012, Olivia around 2015. This may mean that girls' names are more susceptible to fads.

In [86]:
"""
Question 2. How many girls and how many boys were born between 1990 and 2000?
"""
children_1990_2000 = df[df.year.between(1990, 2000, inclusive="both")]
boys_1990_2000 = children_1990_2000[children_1990_2000.sex == "M"]
girls_1990_2000 = children_1990_2000[children_1990_2000.sex == "F"]

boys_total = boys_1990_2000["count"].sum()
girls_total = girls_1990_2000["count"].sum()

difference_percentage = (boys_total / girls_total - 1) * 100

print(
    f"Between years 1990 and 2000 there were born {boys_total} boys and {girls_total} girls.")
print(
    f"That makes boys being born {difference_percentage:.2f} percent more frequently.")


Between years 1990 and 2000 there were born 21437454 boys and 19816748 girls.
That makes boys being born 8.18 percent more frequently.


In [87]:
"""
Question 3. Can you observe the trends in the number of births over all years? Are these trends different between girls and boys? Provide relevant charts.
"""
births_by_year = df[["sex", "count", "year"]].groupby(
    ["year", "sex"], as_index=False).sum()

boys_by_year = births_by_year[births_by_year.sex == "M"][[
    "year", "count"]].reset_index(drop=True).sort_index()
girls_by_year = births_by_year[births_by_year.sex == "F"][[
    "year", "count"]].reset_index(drop=True).sort_index()

total_by_year = boys_by_year + girls_by_year
total_by_year["year"] //= 2

min_values_dict = {"year": [], "count": []}
for i in range(len(boys_by_year)):
    min_value = min(boys_by_year.iloc[i]["count"],
                    girls_by_year.iloc[i]["count"])
    min_values_dict["year"].append(boys_by_year.iloc[i].year)
    min_values_dict["count"].append(min_value)
min_values_by_year = pd.DataFrame(min_values_dict)

fig = go.Figure()
fig.add_trace(go.Scatter(x=min_values_by_year.year, y=min_values_by_year["count"], line=dict(
    color="rgba(0,0,0,0)"), showlegend=False))
fig.add_trace(go.Scatter(x=boys_by_year.year, y=boys_by_year["count"], line=dict(color="#408cff", width=2),
                         name="Boy births", fill="tonexty", fillcolor="rgba(124, 224, 255, 0.5)"))
fig.add_trace(go.Scatter(x=min_values_by_year.year, y=min_values_by_year["count"], line=dict(
    color="rgba(0,0,0,0)"), showlegend=False))
fig.add_trace(go.Scatter(x=girls_by_year.year, y=girls_by_year["count"], line=dict(color="#ff32ad", width=2),
                         name="Girl births", fill="tonexty", fillcolor="rgba(255, 150, 222, 0.5)"))
fig.add_trace(go.Scatter(x=total_by_year.year, y=total_by_year["count"], line=dict(
    color="black", width=1), name="Total births"))
fig.show()


Across the years there can be some trends observed in number of births.
First thing to notice is that there was a relatively low number of new babies born until 1910
and after that year a demographic boom took the birth statistics to whole new levels
as number of new births quadrupled over the next decade.
Then it has stabilized and after a slight decrease caused by Great Depression,
there was another boom in the '50 and '60
that have doubled the amounts from 1920.
After both of these demographic booms we can observe decreases in number of births
but these are not as big as the increases and after time stabilize the statistics
on levels higher than before the boom.
The boom from 1960s is a direct effect of the first one as the 1920 babies were
in their reproduction age during that time.

Those trends are the same for men and women. However, we can observe that in early 20 century
there is slightly more women born and around 1937 this trend turns over and since then, there are men born.

In [88]:
"""
Question 4. Comment on statistics for your name. Give as meny insights as possible.
Paweł -> Paul
Maksymilian/Maks -> Maximilian/Max
"""
pauls = df.loc[(df.name == "Paul") & (df.sex == "M")].sort_values(
    by="year").reset_index(drop=True)

fig = px.line(pauls, x="year", y="count", color="name",
              labels=("Paul"), title="Boys named Paul")
fig.show()

The increases in the number of boys named Paul aligns pretty well with the two demographic booms
described in the previous question. Although there is one exception which is a peak in 1963 and 1964
that might be correlated with formation of The Beatles music band and the fact that one of the
members' name is Paul McCartney.
After that, the name started to loose on popularity and had a major decline in early 1970s,
then stabilized for a couple of years and started falling down back again after 1980.
The number of Pauls born in 2018 was almost the same as in 1909 despite the fact that
overall number of births was 7 times as high.

In [89]:
# maximilian = df.loc[(df.name == "Maximilian") & (df.sex == "M")].sort_values(by="year").reset_index(drop=True)
max = df.loc[(df.name == "Max") & (df.sex == "M")].sort_values(
    by="year").reset_index(drop=True)
fig = go.Figure()
# fig = px.line(maximilian, x="year", y="count", color="name", labels=("Maximilian"), title="Boys named Maximilian")
fig.add_trace(go.Scatter(x=max['year'], y=max["count"], line=dict(color="#ff000f", width=2),
                         name="Max"))
fig.add_trace(go.Scatter(x=total_by_year.year,
              y=boys_by_year["count"]/1000, line=dict(color="black", width=1), name="Total boys births/1000"))
fig.show()

The number of boys named Max incrased with demographic boom between 1910-1920.
After that period popularity of the name decreased until 80' when number of baby Maxes skyrocked.
Probably this was due to premiere of two films from Max Mad series in 1979 and 1985.
Since then popularity of name was increasing until 2011 when it began to decrease.